# Download daily price from yfinance

* Truncate and update px table for all symbols in S table. yf_symbol required

In [65]:
import os
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine

sql_username = os.environ.get('SQL_USERNAME')
sql_password = os.environ.get('SQL_PASSWORD')
sql_host = os.environ.get('SQL_HOST')
sql_database = os.environ.get('SQL_DATABASE')
engine = create_engine(f'mysql://{sql_username}:{sql_password}@{sql_host}/{sql_database}')

sql_query = 'select yf_symbol from s;'
tickers = pd.read_sql(sql_query, engine).squeeze().tolist()
data = yf.download(tickers, start=None, end=None, interval='1d')
df = pd.DataFrame(data)

# reshape df. Current columns are (open, ticker), (close, ticker), (high, ticker), etc
melted_df = df.melt(ignore_index=False).reset_index()
reshaped_df = melted_df.pivot(index=['Date','variable_1'], columns='variable_0', values='value')
reshaped_df.reset_index(inplace=True)
reshaped_df.rename(columns={'Date':'dt','variable_1':'symbol','Adj Close':'adj_close','Close':'close','High':'high','Low':'low','Open':'open','Volume':'vol'}, inplace=True)

# create temp table to hold px table data
with engine.connect() as conn:
    conn.execute('create temporary table px_temp like px')
    conn.execute('insert into px_temp select * from px')

# update the px table
try:
    with engine.connect() as conn:
        conn.execute('truncate table px')
        reshaped_df.to_sql(name='px', con=conn, if_exists='append', index=False)
        
    # if everything is ok, drop temp table
    print('Data updated successfully..')
    with engine.connect() as conn:
        conn.execute('drop temporary table px_temp')

except Exception as e:
    # If anything goes wrong, restore the px table
    print('Error uploading data:', e)
    with engine.connect() as conn:
        conn.execute('truncate table px')
        conn.execute('insert into px select * from px_temp')
        conn.execute('drop temporary table px_temp')
    print('Data retored to the previous status..')
    


[*********************100%***********************]  39 of 39 completed
Data updated successfully..
